In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import os, sys
import cv2
import PIL
import time
from skimage import color
import os.path
from PIL import Image, ImageOps

In [2]:
import time
import os, sys, os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, MaxPool2D, MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import confusion_matrix, classification_report
import itertools

In [3]:
cwd = os.getcwd()
# os.listdir(cwd)

In [4]:
training_paths = ["./train_tumor/Tumor/", "./train_tumor/NoTumor/"]
validation_paths = ["./validation_tumor/Tumor/", "./validation_tumor/NoTumor/"]
test_paths = ["./test_tumor/Tumor/", "./test_tumor/NoTumor/"]

In [5]:
# Resizing an saving the training images in same folder with same file name

for path in training_paths:
    for img_name in os.listdir(path):
        f_img = path+img_name
        img = Image.open(f_img)
        #img_gray = img.convert('L')
        img_gray = ImageOps.grayscale(img)
        try:
            img_gray = img_gray.resize((300,300), PIL.Image.ANTIALIAS)
            img_gray.save(f_img, quality=95)
        except ValueError:
            print("There is something wrong")

In [6]:
# Resizing an saving the training images in same folder with same file name

for path in validation_paths:
    for img_name in os.listdir(path):
        f_img = path+img_name
        img = Image.open(f_img)
        #img_gray = img.convert('L')
        img_gray = ImageOps.grayscale(img)
        try:
            img_gray = img_gray.resize((300,300), PIL.Image.ANTIALIAS)
            img_gray.save(f_img, quality=95)
        except ValueError:
            print("There is something wrong")

In [7]:
# Resizing an saving the test images in same folder with same file name

for path in test_paths:
    for img_name in os.listdir(path):
        f_img = path+img_name
        img = Image.open(f_img)
        #img_gray = img.convert('L')
        img_gray = ImageOps.grayscale(img)
        try:
            img_gray = img_gray.resize((300,300), PIL.Image.ANTIALIAS)
            img_gray.save(f_img, quality=95)
        except ValueError:
            print("There is something wrong")

In [8]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train_tumor",
                                          target_size=(300,300),
                                          batch_size = 32,
                                          class_mode = 'binary', color_mode="grayscale")

validation_dataset = train.flow_from_directory("validation_tumor",
                                          target_size=(300,300),
                                          batch_size = 32,
                                          class_mode = 'binary', color_mode="grayscale")

test_dataset = test.flow_from_directory("test_tumor",
                                          target_size=(300,300),
                                          batch_size =32,
                                          class_mode = 'binary', color_mode="grayscale")

Found 88 images belonging to 2 classes.
Found 44 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [9]:
validation_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [10]:
test_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [11]:
train_dataset.class_indices

{'NoTumor': 0, 'Tumor': 1}

In [12]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(300,300,1)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
#steps_per_epoch = train_imagesize/batch_size

start = time.time()

model.fit(train_dataset,
         epochs = 20, batch_size=10,
         validation_data = validation_dataset  
         )

end = time.time()
print(end - start)

Epoch 1/20
3/3 [==============================] - 13s 4s/step - loss: 0.7380 - accuracy: 0.4773 - val_loss: 0.4495 - val_accuracy: 0.8409
Epoch 2/20
3/3 [==============================] - 11s 4s/step - loss: 0.4425 - accuracy: 0.8295 - val_loss: 0.3034 - val_accuracy: 0.8182
Epoch 3/20
3/3 [==============================] - 11s 4s/step - loss: 0.3324 - accuracy: 0.8523 - val_loss: 0.5935 - val_accuracy: 0.8636
Epoch 4/20
3/3 [==============================] - 11s 4s/step - loss: 0.2519 - accuracy: 0.9091 - val_loss: 0.3229 - val_accuracy: 0.8182
Epoch 5/20
3/3 [==============================] - 11s 3s/step - loss: 0.2105 - accuracy: 0.9318 - val_loss: 0.3192 - val_accuracy: 0.8636
Epoch 6/20
3/3 [==============================] - 11s 4s/step - loss: 0.1783 - accuracy: 0.9432 - val_loss: 0.3985 - val_accuracy: 0.8636
Epoch 7/20
3/3 [==============================] - 11s 4s/step - loss: 0.1408 - accuracy: 0.9545 - val_loss: 0.2882 - val_accuracy: 0.8636
Epoch 8/20
3/3 [==================

In [15]:
model.evaluate(test_dataset)

1/1 [==============================] - 1s 617ms/step - loss: 0.1529 - accuracy: 0.9091


[0.15294446051120758, 0.9090909361839294]

In [20]:
# Specifying Model Architecture (Convolutional Neural Network)
early_stopping_monitor = EarlyStopping(patience=3)

model3 = Sequential()

# Convolutional layer, Batch Normalization layer, and maxpool layer 1
model3.add(Conv2D(32,(3,3),activation='relu',input_shape=(300,300,1)))
model3.add(BatchNormalization())
model3.add(MaxPool2D(2,2))

# Convolutional layer,  Batch Normalization layer, and maxpool layer 2
model3.add(Conv2D(64,(3,3),activation='relu'))
# model3.add(BatchNormalization())
model3.add(MaxPool2D(2,2))

# Convolutional layer,  Batch Normalization layer, and maxpool layer 3
model3.add(Conv2D(128,(3,3),activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPool2D(2,2))

# Convolutional layer,  Batch Normalization layer, and maxpool layer 4
model3.add(Conv2D(256,(3,3),activation='relu'))
# model3.add(BatchNormalization())
model3.add(MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model3.add(Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model3.add(Dense(512,activation='relu'))
# model3.add(Dropout(0.25))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model3.add(Dense(1,activation='sigmoid'))

In [21]:
model3.compile(optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'), loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
#steps_per_epoch = train_imagesize/batch_size

start = time.time()

model3.fit(train_dataset, epochs = 25, batch_size=32, validation_data = validation_dataset, callbacks = [early_stopping_monitor])

end = time.time() 
print(end - start)

Epoch 1/25
3/3 [==============================] - 8s 3s/step - loss: 41.7471 - accuracy: 0.5682 - val_loss: 0.6768 - val_accuracy: 0.5000
Epoch 2/25
3/3 [==============================] - 7s 3s/step - loss: 14.9527 - accuracy: 0.5909 - val_loss: 0.6549 - val_accuracy: 0.6591
Epoch 3/25
3/3 [==============================] - 8s 3s/step - loss: 2.0088 - accuracy: 0.8977 - val_loss: 0.7205 - val_accuracy: 0.5000
Epoch 4/25
3/3 [==============================] - 7s 3s/step - loss: 1.9014 - accuracy: 0.9091 - val_loss: 1.0101 - val_accuracy: 0.5000
Epoch 5/25
3/3 [==============================] - 7s 3s/step - loss: 1.1662 - accuracy: 0.9432 - val_loss: 1.3005 - val_accuracy: 0.5000
37.89298748970032


In [ ]:
print(model_300)
print(np.mean(model_300))

In [ ]:
#steps_per_epoch = train_imagesize/batch_size
result = 0
model_300 = []
start = time.time()
for i in range(3):
    model.fit(train_dataset,
             epochs = 5, batch_size=10,
             validation_data = 0.0  
             )
    result = model.evaluate(test_dataset)
    model_300.append(result[1])
end = time.time()
print(end - start)

In [ ]:
# Saving model file
import pickle
pickle.dump(result, open('model.pkl', 'wb'))

In [ ]:
# Loading model file again
with open('model.pkl', 'rb') as file:
    pickle_model = pickle.load(file)

In [ ]:
# Inspecting model file
print(type(pickle_model))
print(pickle_model)

In [ ]:
# Predicting single image
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (300, 300, 1))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('./test_tumor4/Tumor/Tr-me_0076.jpg')
model.predict(image)

In [ ]:
# Predicting single image
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (300, 300, 1))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('./test_tumor4/NoTumor/Tr-no_0054.jpg')
model.predict(image)

In [ ]:
dd = os.listdir("./train_tumor4/Tumor/")
rgb = plt.imread('./train_tumor4/Tumor/Tr-me_0010.jpg')
plt.imshow(rgb)

In [ ]:
"""

#steps_per_epoch = train_imagesize/batch_size

model.fit_generator(train_dataset,
         steps_per_epoch = 250,
         epochs = 10,
         validation_data = test_dataset
       
         )
         
"""

In [ ]:
"""

# Manually Predicting 
test_path = '../test_tumor'
Tumor=0
NoTumor=0
for i in os.listdir(test_path):
    img = image.load_img(test_path + "//" +i, target_size=(512,512,3))
    plt.imshow(img)
    plt.show()
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    if val == 0:
        NoTumor +=1
        print('NoTumor')
    else:
        Tumor +=1
        print('Tumor')
print(f'Tumor: {Tumor}, NoTumor: {NoTumor}')


"""

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
rounded_predictions = np.argmax(predictions, axis=-1)

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['no_side_effects','had_side_effects']

In [ ]:
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')